### This notebook prepares the downloaded historical stock data and calculates futures necessary to run predictions

Import libraries

In [1]:
import sys
sys.path.append('/helpers')
from notebook_widgets.data_prep import selectors, prep_features_button, prep_features_button_output
# display widgets
for s in selectors:
    display(selectors[s]['display'])
    
# display buttons
display(prep_features_button, prep_features_button_output)

Dropdown(description='File:', options=('download_20221209_181319', 'download_20221209_181333', 'download_20221…

Button(description='Prepare features', style=ButtonStyle())

Output()

Model target and features

| Target | Description |
| :--- | :--- |
| `adj_close_next_day` | adjusted closing price increase on the next day |


| Feature | Description |
| :--- | :--- |
| `highest_5`| Is the price highest in 5 days? |
| `highest_10`| Is the price highest in 10 days? |
| `highest_30`| Is the price highest in 30 days? |
| `highest_ever`| Is the price highest ever seen? |
| `is_monday`| Is it a Monday? |
| `is_friday`| Is it a Friday? |
| `increase_streak`| Number of consecutive days the close price has been increasing, i.e. higher than the day before|
| `decline_streak`| Number of consecutive days the close price has been decreasing, i.e. lower than the day before|
| `previous_swing`| Difference between the high and low price for the previous day|